## Import Packages
---

In [12]:
import os
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import scipy
from PIL import Image
import pandas as pd
import cv2
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.python.framework import ops

## Load Data
---

In [5]:
palm=pd.read_csv('palmtree_annotations.csv')
palm["Label"].iloc[576]='palmtree'
palm["Label"]=palm["Label"].str.replace('nope','0').str.replace('palmtree','1')
palm = palm.iloc[:-1 , :]

In [13]:
Y=palm["Label"].to_list()
X= np.array([cv2.imread(file).tolist() for file in palm["Image_name"].to_list()])
X_flatten =X.reshape(X.shape[0],-1).T
Y_flatten=np.reshape(np.array(Y),(1,len(Y)))
palm_shuffled=palm.sample(frac=1).reset_index(drop=True)

In [55]:
trainset_size=int(len(palm_shuffled)*0.7)

In [56]:
train, test = palm_shuffled.iloc[:trainset_size], palm_shuffled.iloc[trainset_size:]

In [57]:
train

,Image_name,Label
0,Bilder\20190930_073643_HDR.jpg,0
1,Bilder\IMG_20190401_201715_626.jpg,0
2,Bilder\IMG_1386.JPG,0
3,Bilder\IMG_20190825_174632.jpg,0
4,Bilder\FB_IMG_1577263574272.jpg,0
...,...,...
452,Bilder\IMG-20200825-WA0009.jpg,0
453,Bilder\20374544_1662578057116554_1637137420869...,0
454,Bilder\IMG_20180331_064103_460.jpg,0
455,Bilder\IMG_20200904_085752.jpg,0


In [58]:
test

,Image_name,Label
457,Bilder\IMG_20180427_062546.jpg,0
458,Bilder\IMG_20170816_105223_150.jpg,0
459,Bilder\IMG_20181120_224742_614.jpg,0
460,Bilder\IMG_20200120_144623.jpg,1
461,Bilder\IMG_20160411_221114.jpg,0
...,...,...
648,Bilder\FB_IMG_1455783195460.jpg,0
649,Bilder\IMG_1323.JPG,0
650,Bilder\IMG_1815.JPG,0
651,Bilder\20190930_074642_HDR.jpg,0


In [29]:
#Bilder laden..
Ytrain=train["Label"].to_list()
train_y=np.reshape(np.array(Ytrain,dtype=float),(1,len(Ytrain)))
Xtrain= np.array([cv2.imread(file).tolist() for file in train["Image_name"].to_list()])
#train_x =Xtrain.reshape(Xtrain.shape[0],-1).T
Ytest=test["Label"].to_list()
test_y=np.reshape(np.array(Ytest,dtype=float),(1,len(Ytest)))
Xtest= np.array([cv2.imread(file).tolist() for file in test["Image_name"].to_list()])
#test_x =Xtest.reshape(Xtest.shape[0],-1).T

In [48]:
# Normalise image vectors
X_train = Xtrain/255.
X_test = Xtest/255.

# Reshape
Y_train = train_y.T
Y_test = test_y.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 457
number of test examples = 196
X_train shape: (457, 100, 100, 3)
Y_train shape: (457, 1)
X_test shape: (196, 100, 100, 3)
Y_test shape: (196, 1)


## Define Model
---

In [39]:
def palmModel():
    model = tf.keras.Sequential([

            tfl.ZeroPadding2D(padding=3, input_shape=(100,100,3)),
            tfl.Conv2D(filters=32, kernel_size=(7, 7), strides=1),
            tfl.BatchNormalization(axis=3),
            tfl.ReLU(),
            tfl.MaxPool2D(),
            tfl.Flatten(),
            tfl.Dense(units=1, activation='sigmoid')
        ])
  
    return model

In [40]:
palm_model=palmModel()
palm_model.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

In [41]:
palm_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 106, 106, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 100, 100, 32)      4736      
_________________________________________________________________
batch_normalization (BatchNo (None, 100, 100, 32)      128       
_________________________________________________________________
re_lu (ReLU)                 (None, 100, 100, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 80000)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 8

## Fit Model
---

In [49]:
palm_model.fit(X_train, Y_train, epochs=10, batch_size=16)

Epoch 1/10
29/29 [==============================] - 6s 202ms/step - loss: 5.6530 - accuracy: 0.8140
Epoch 2/10
29/29 [==============================] - 6s 198ms/step - loss: 0.9962 - accuracy: 0.8337
Epoch 3/10
29/29 [==============================] - 6s 207ms/step - loss: 1.3655 - accuracy: 0.8446
Epoch 4/10
29/29 [==============================] - 6s 203ms/step - loss: 0.6034 - accuracy: 0.8884
Epoch 5/10
29/29 [==============================] - 6s 200ms/step - loss: 0.3853 - accuracy: 0.9212
Epoch 6/10
29/29 [==============================] - 6s 207ms/step - loss: 0.3608 - accuracy: 0.9234
Epoch 7/10
29/29 [==============================] - 6s 201ms/step - loss: 0.1938 - accuracy: 0.9387
Epoch 8/10
29/29 [==============================] - 6s 207ms/step - loss: 0.0914 - accuracy: 0.9672
Epoch 9/10
29/29 [==============================] - 6s 205ms/step - loss: 0.1021 - accuracy: 0.9606
Epoch 10/10
29/29 [==============================] - 6s 199ms/step - loss: 0.1595 - accuracy: 0.9694